__Word Alignment Assignment__

Your task is to learn word alignments for the data provided with this Python Notebook. 

Start by running the 'train' function below and implementing the assertions which will fail. Then consider the following improvements to the baseline model:
* Is the TranslationModel parameterized efficiently?
* What form of PriorModel would help here? (Currently the PriorModel is uniform.)
* How could you use a Hidden Markov Model to model word alignment indices? (There's an implementation of simple HMM below to help you start.)
* How could you initialize more complex models from simpler ones?
* How could you model words that are not aligned to anything?

Grades will be assigned as follows*:

 AER below on blinds   |  Points 
----------|-------------
 0.40 - 0.50 |   3 
 0.35 - 0.40 |   6 
 0.30 - 0.35 |  9    
 0.25 - 0.30 |  12    
 0.00 - 0.25 |  15   
 
You should save the notebook with the final scores for 'dev' and 'test' test sets.



In [134]:
# This cell contains the generative models that you may want to use for word alignment.
# Currently only the TranslationModel is at all functional.

import numpy as np
from collections import defaultdict

class TranslationModel:
    "Models conditional distribution over trg words given a src word."

    def __init__(self, src_corpus, trg_corpus):
        self._trg_given_src_probs = defaultdict(lambda : defaultdict(lambda : 0.0001))
        self._src_trg_counts = defaultdict(lambda : defaultdict(lambda : 0.0))
        self.get_init_counts(src_corpus, trg_corpus)
        
    def get_init_counts(self, src_corpus, trg_corpus):
        for src_tokens, trg_tokens in zip(src_corpus, trg_corpus):
            for src_token in src_tokens:
                for trg_token in trg_tokens:
                    self._src_trg_counts[src_token][trg_token] += 1

        self.recompute_parameters()        

    def get_params(self):
        return self._trg_given_src_probs

    def get_conditional_prob(self, src_token, trg_token):
        "Return the conditional probability of trg_token given src_token."
        return self._trg_given_src_probs[src_token][trg_token]

    def get_parameters_for_sentence_pair(self, src_tokens, trg_tokens):
        "Returns matrix with t[i][j] = p(f_j|e_i)."
        return np.array([[self._trg_given_src_probs[trg_token][src_token]
                          for trg_token in trg_tokens] for src_token in src_tokens])
        
    def collect_statistics(self, src_tokens, trg_tokens, posterior_matrix):
        "Accumulate counts of translations from: posterior_matrix[i][j] = p(a_j=i|e, f)"
        assert posterior_matrix.shape == (len(src_tokens), len(trg_tokens))
        
        for i in range(posterior_matrix.shape[0]):
            for j in range(posterior_matrix.shape[1]):
                self._src_trg_counts[src_tokens[i]][trg_tokens[j]] += posterior_matrix[i, j]
#         print(self._src_trg_counts)
        
    def recompute_parameters(self):
        "Reestimate parameters and reset counters."
        self._trg_given_src_probs = defaultdict(lambda : defaultdict(lambda : 0.0001))
        
        for src in self._src_trg_counts.keys():
            for trg in self._src_trg_counts[src].keys():
                self._trg_given_src_probs[trg][src] += self._src_trg_counts[src][trg]
                
        
        for trg in self._trg_given_src_probs.keys():
            ss = sum(self._trg_given_src_probs[trg].values())
            for src in self._trg_given_src_probs[trg].keys():
                self._trg_given_src_probs[trg][src] = (self._trg_given_src_probs[trg][src]) / ss
            
#         print({ s: {e:  self._trg_given_src_probs[s][e] for e in self._trg_given_src_probs[s].keys() } for s in self._trg_given_src_probs.keys()})
        self._src_trg_counts = defaultdict(lambda : defaultdict(lambda : 0.0))

class PriorModel:
    "Models the prior probability of an alignment given only the sentence lengths and token indices."

    def __init__(self, src_corpus, trg_corpus):
        "Add counters and parameters here for more sophisticated models."
        self._distance_counts = defaultdict(lambda : defaultdict(lambda : defaultdict(lambda : 0.0)))
        self._distance_probs = defaultdict(lambda : defaultdict(lambda : defaultdict(lambda : 0.0001)))

    def get_parameters_for_sentence_pair(self, src_length, trg_length):
        pair = (src_length, trg_length)
        if pair not in self._distance_probs:
            return np.ones((src_length, trg_length)) / src_length
        
        return np.array([[self._distance_probs[pair][trg][src]
                          for trg in range(trg_length)] for src in range(src_length)])
    
    def get_prior_prob(self, src_index, trg_index, src_length, trg_length):
        "Returns a uniform prior probability."
        return 1.0 / src_length

    def collect_statistics(self, src_length, trg_length, posterior_matrix):
        "Extract the necessary statistics from this matrix if needed."
#         return
        pair = (src_length, trg_length)
        for i in range(src_length):
            for j in range(trg_length):
                self._distance_counts[pair][i][j] += posterior_matrix[i, j] * 1000

    def recompute_parameters(self):
        "Reestimate the parameters and reset counters."
#         return
        self._distance_probs =  defaultdict(lambda : defaultdict(lambda : defaultdict(lambda : 0.0001)))

        for len_pair in self._distance_counts.keys():
            for src in self._distance_counts[len_pair].keys():
                for trg in self._distance_counts[len_pair][src].keys():
                    self._distance_probs[len_pair][trg][src] += self._distance_counts[len_pair][src][trg]
                    
        for len_pair in self._distance_probs.keys():
            for trg in self._distance_probs[len_pair].keys():
                ss = sum(self._distance_probs[len_pair][trg].values())
                for src in self._distance_probs[len_pair][trg].keys():
                    self._distance_probs[len_pair][trg][src] /= ss

        self._distance_counts =  defaultdict(lambda : defaultdict(lambda : defaultdict(lambda : 0.0)))


class TransitionModel:
    "Models the prior probability of an alignment conditioned on previous alignment."

    def __init__(self, src_corpus, trg_corpus):
        "Add counters and parameters here for more sophisticated models."
        self._counter = {}
        self._prob = {}

    def get_parameters_for_sentence_pair(self, src_length):
        "Retrieve the parameters for this sentence pair: A[k, i] = p(a_{j} = i|a_{j-1} = k)"
        pass

    def collect_statistics(self, src_length, bigram_posteriors):
        "Extract statistics from the bigram_posterior[i][j]: p(a_{t-1} = i, a_{t} = j| e, f)"
        pass
        
    def recompute_parameters(self):
        "Recompute the transition matrix"
        pass


In [144]:
# This cell contains the framework for training and evaluating a model using EM.

from utils import read_parallel_corpus, extract_test_alignments, score_alignments, write_aligned_corpus

def get_posterior(emission_prob):
    ss = np.sum(emission_prob, axis=1, keepdims=True)
    post = emission_prob / ss ## check this
    
#     post = np.zeros(emission_prob.shape)
#     for j in range(post.shape[1]):
#         s = np.sum(emission_prob[:, j])
#         for i in range(post.shape[0]):
#             post[i, j] = emission_prob[i, j] / s

    return post

def get_posterior_comp(prior_prob, emission_prob):
    tt = emission_prob * prior_prob
    ss = np.sum(tt, axis=1, keepdims=True)
    post = tt / ss ## check this
    
    return post

def compute_log_likelihood(posterior, emission):
#     locs = posterior > 0.0001
#     zers = np.zeros_like(posterior)
#     log_likelihood = np.sum(np.where(locs,  posterior * np.log(emission + 0.0001), zers))
#     return log_likelihood
    return (posterior * np.log(emission)).sum()

def compute_log_likelihood_comp(prior, posterior, emission):
    return (posterior * np.log(emission)).sum()

def infer_posteriors(src_tokens, trg_tokens, prior_model, translation_model):
    "Compute the posterior probability p(a_j=i | f, e) for each target token f_j given e and f."
    # P[i][j] = P(a_j=i| I, J): prior alignment prob given only sentence lengths.
    P = prior_model.get_parameters_for_sentence_pair(len(src_tokens), len(trg_tokens))
    # Emissions prob of seeing f_j given e_i: T[i][j] = P(f_j|e_i)
    T = translation_model.get_parameters_for_sentence_pair(src_tokens, trg_tokens) 
    # posterior[i][j] = P(a_j = i | e, f)
#     posteriors = get_posterior(T)
#     log_likelihood = compute_log_likelihood(posteriors, T)    

    posteriors = get_posterior_comp(P, T)
    log_likelihood = compute_log_likelihood_comp(P, posteriors, T)    

    return posteriors, log_likelihood

def collect_expected_statistics(src_corpus, trg_corpus, prior_model, translation_model):
    "E-step: infer posterior distribution over each sentence pair and collect statistics."
    corpus_log_likelihood = 0.0
    for src_tokens, trg_tokens in zip(src_corpus, trg_corpus):
        # Infer posterior
        posteriors, log_likelihood = infer_posteriors(src_tokens, trg_tokens, prior_model, translation_model)
        # Collect statistics in each model.
#         print('post', posteriors)
        prior_model.collect_statistics(len(src_tokens), len(trg_tokens), posteriors)
        translation_model.collect_statistics(src_tokens, trg_tokens, posteriors)
        # Update log prob
        
        corpus_log_likelihood += log_likelihood
    return corpus_log_likelihood

def estimate_models(src_corpus, trg_corpus, prior_model, translation_model, num_iterations):
    "Estimate models iteratively using EM."
    for iteration in range(num_iterations):
        # E-step
        corpus_log_likelihood = collect_expected_statistics(src_corpus, trg_corpus, prior_model, translation_model)
        # M-step
        prior_model.recompute_parameters()
        translation_model.recompute_parameters()
        print("corpus log likelihood: %1.3f" % corpus_log_likelihood)
    return prior_model, translation_model

def get_alignments_from_posterior(posteriors):
    "Returns the MAP alignment for each target word given the posteriors."
    alignments = {}
    for src_index, trg_index in enumerate(np.argmax(posteriors, 0)):
        if src_index not in alignments:
            alignments[src_index] = {}
        alignments[src_index][trg_index] = '*'
    return alignments

def align_corpus(src_corpus, trg_corpus, prior_model, translation_model):
    "Align each sentence pair in the corpus in turn."
    aligned_corpus = []
    for src_tokens, trg_tokens in zip(src_corpus, trg_corpus):
        posteriors, _ = infer_posteriors(src_tokens, trg_tokens, prior_model, translation_model)
        alignments = get_alignments_from_posterior(posteriors)
        aligned_corpus.append((src_tokens, trg_tokens, alignments))
    return aligned_corpus

def initialize_models(src_corpus, trg_corpus):
    prior_model = PriorModel(src_corpus, trg_corpus)
    translation_model = TranslationModel(src_corpus, trg_corpus)
    return prior_model, translation_model

def normalize(src_corpus, trg_corpus):
    import nltk
    lemm = nltk.stem.WordNetLemmatizer()
    # Consider applying some normalization here to reduce the numbers of parameters.
    src_lemm = []
    trg_lemm = []
    for src_tokens in src_corpus:
        src_lemm.append(np.array([token.lower()[:4] for token in src_tokens]))
            
#     for src_tokens in src_corpus:
#         src_tagged.append(np.array([tag for token, tag in nltk.pos_tag(src_tokens)]))
    
    
    for tokens in trg_corpus:
        trg_lemm.append(np.array([token.lower()[:4] for token in tokens]))
    
    return src_lemm, trg_lemm

def train(num_iterations):
    # Load data
    src_corpus, trg_corpus, _ = read_parallel_corpus('en-cs.first5K')
    # Normalize it.
    src_corpus, trg_corpus = normalize(src_corpus, trg_corpus)
    # Initialize models
    print('start init')
    prior_model, translation_model = initialize_models(src_corpus, trg_corpus)
    # Train them with EM
    print('start estimate')
    prior_model, translation_model = estimate_models(src_corpus, trg_corpus, prior_model, translation_model, num_iterations)    
    # Align the corpus
    aligned_corpus = align_corpus(src_corpus, trg_corpus, prior_model, translation_model)
    # Return alignments on test sets.
    return extract_test_alignments(aligned_corpus)

def evaluate(test_set_alignments):
    src_dev, trg_dev, wa_dev = read_parallel_corpus('en-cs-wa.dev', has_alignments=True)
#     src_test, trg_test, wa_test = read_parallel_corpus('en-cs-wa.test', has_alignments=True)
    print('recall %1.3f; precision %1.3f; aer %1.3f' % score_alignments(wa_dev, test_set_alignments['dev']))
#     print('recall %1.3f; precision %1.3f; aer %1.3f' % score_alignments(wa_test, test_set_alignments['test']))            

Start by running the following cell and fixing some failures:

In [145]:
test_set_alignments = train(10)
evaluate(test_set_alignments)

start init
start estimate
corpus log likelihood: -855373.521
corpus log likelihood: -354279.246
corpus log likelihood: -259138.145
corpus log likelihood: -178753.889
corpus log likelihood: -135180.515
corpus log likelihood: -114138.367
corpus log likelihood: -103643.470
corpus log likelihood: -97810.552
corpus log likelihood: -94208.759
corpus log likelihood: -91821.345
{0: {0: 'S'}, 3: {1: 'S'}, 9: {10: 'S', 9: 'P'}, 10: {11: 'P', 15: 'S'}, 12: {11: 'P', 12: 'P'}, 13: {12: 'S'}, 14: {12: 'S'}, 15: {12: 'S'}, 11: {13: 'S', 14: 'P'}, 16: {16: 'S'}, 1: {2: 'S'}, 4: {3: 'S', 4: 'S'}, 5: {5: 'P', 7: 'S'}, 6: {6: 'P'}, 7: {6: 'S'}, 8: {8: 'S'}} {0: {3: '*'}, 1: {2: '*'}, 2: {5: '*'}, 3: {1: '*'}, 4: {11: '*'}, 5: {7: '*'}, 6: {5: '*'}, 7: {6: '*'}, 8: {8: '*'}, 9: {10: '*'}, 10: {12: '*'}, 11: {13: '*'}, 12: {11: '*'}, 13: {5: '*'}, 14: {11: '*'}, 15: {14: '*'}, 16: {16: '*'}}
{0: {0: 'S'}, 1: {1: 'S'}, 11: {10: 'S'}, 10: {11: 'S'}, 14: {12: 'P', 13: 'S'}, 15: {14: 'P'}, 16: {14: 'S'}, 18: 

In [146]:
# test_set_alignments